In [7]:
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install einops
!pip install safetensors
!pip install torch
!pip install xformers
!pip install streamlit
!pip install PyPDF2
!pip install langchain

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-3v5gxjxz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-3v5gxjxz
  Resolved https://github.com/huggingface/transformers.git to commit 6d49b9dcbff5af73fcbcfc79ee0e83d0c56983e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-yti4rtnm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-yti4rtnm
  Resolved https://github.com/huggingface/peft.git to commit 93d0c03d5ba6b2a6b16b7ca887e740a67bc680f3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-a5ctk304
  Runni

In [13]:
import os
import base64
import streamlit as st
import PyPDF2
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain

In [14]:


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [15]:
model_id = 'vilsonrodrigues/falcon-7b-instruct-sharded'

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id, 
        device_map="auto",
        quantization_config=quantization_config,
        )

tokenizer = AutoTokenizer.from_pretrained(model_id)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [17]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=296,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [18]:
from langchain import PromptTemplate, LLMChain

llm = HuggingFacePipeline(pipeline=pipeline)

template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(
    template=template, 
    input_variables= ["question"]
)

llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain("You are aissistant whose goal is to rank resumes on the basis of the job description")

The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.


{'question': 'You are aissistant whose goal is to rank resumes on the basis of the job description',
 'text': ' First, determine how many positions there are. Then, for each position, identify the key criteria (for example, experience, education, skills, etc.). Finally, rank each resume according to these criteria and use the final ranking to prioritize which resumes should be reviewed next.'}

KeyboardInterrupt: 

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501